## Start HumemAI

In [ ]:
from humemai.utils import disable_logger
disable_logger()

from gremlin_python.structure.graph import Graph, Vertex, Edge
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction

import json
from humemai.janusgraph import Humemai
from tqdm.auto import tqdm

humemai = Humemai()
humemai.start_containers()
humemai.connect()
# humemai.remove_all_data()


with open("./raw-data.json") as f:
    data = json.load(f)

# humemai.disconnect()
# humemai.stop_containers()
# humemai.remove_containers()

from humemai.utils import disable_logger
disable_logger()

In [ ]:
humemai.remove_all_data()

for data_point in tqdm(data):
    properties = {
        "cp_num": data_point["cp_num"],
        "participant_num": data_point["participant"],
        "cp_name": data_point["cp_name"],
        "ticks_lasted": data_point["ticks_lasted"],
        "round_num": data_point["round_num"],
        "event_time": data_point["timestamp"],
        "remaining_time": data_point["remaining_time"],
        "remaining_rocks": data_point["remaining_rocks"],
        "victim_harm": data_point["victim_harm"],
    }
    time_vertex = humemai.write_time_vertex(data_point["timestamp"])

    robot_vertex = humemai.write_long_term_vertex("robot", "episodic", time_vertex)
    cp_label = "cp_num_" + str(data_point["cp_num"])
    cp_vertex = humemai.write_long_term_vertex(
        cp_label, "episodic", time_vertex, properties
    )
    humemai.write_edge(robot_vertex, "has_cp", cp_vertex)

    participant_label = "participant_" + str(data_point["participant"])
    participant_vertex = humemai.write_long_term_vertex(
        participant_label, "episodic", time_vertex
    )
    humemai.write_edge(participant_vertex, "has_cp", cp_vertex)

    situation = [bar for foo in data_point["situation"] for bar in foo]
    
    if situation:        
        situation_label = "situation_" + str(data_point["cp_num"])
        situation_vertex = humemai.write_long_term_vertex(
            situation_label, "episodic", time_vertex, {foo["type"]: foo["content"] for foo in situation}
        )
        humemai.write_edge(cp_vertex, "has_situation", situation_vertex)

        human_action = [bar for foo in data_point["HumanAction"] for bar in foo]
        robot_action = [bar for foo in data_point["RobotAction"] for bar in foo]

        # for idx, action in enumerate(data_point["HumanAction"]):
        #     action_label = "action_" + str(idx)
        #     action_vertex = humemai.write_long_term_vertex(
        #         action_label, "episodic", time_vertex, action
        #     )
        #     humemai.write_edge(participant_vertex, "has_action", action_vertex)

# Merge the duplicate nodes
humemai.merge_vertices_with_same_label("robot")

vertices = humemai.get_vertices_by_partial_label("participant")
for vertex in vertices:
    humemai.merge_vertices_with_same_label(vertex.label)

In [ ]:
action

In [ ]:
{foo["type"]: foo["content"] for foo in situation}

In [ ]:
situation

In [ ]:
for action in data_point["HumanAction"]:
    print(action)

In [ ]:
from collections import Counter

# Get all vertex labels
labels = humemai.g.V().label().toList()

# Count the occurrences of each label
label_counts = Counter(labels)

# Filter labels that occur more than once
labels_more_than_once = {label: count for label, count in label_counts.items() if count > 1}

print(labels_more_than_once)


In [ ]:
for foo in labels_more_than_once:
    print(foo)

In [ ]:
len(humemai.g.V().label().toList()), len(set(humemai.g.V().label().toList()))